In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl


import sys
sys.path.append('../Utilities/')

from tqdm.notebook import tqdm

import importlib
import data_utils
importlib.reload(data_utils)

## Import MDS from sklearn
from sklearn.manifold import MDS
from sklearn.metrics import accuracy_score, f1_score
mds = MDS(n_components=1, random_state=0, normalized_stress='auto')

In [2]:
class UNet1D(nn.Module):
    def __init__(self, in_channels, out_channels, depth=2, num_layers=2):
        super(UNet1D, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_layers = num_layers
        self.depth = depth
        self.encoder = nn.ModuleList()
        self.decoder = nn.ModuleList()
        self.num_start_filters = 32

        self._create_unet(self.in_channels, self.num_start_filters)
        self.bottleneck = nn.Sequential(
            nn.Conv1d(self.num_start_filters * 2 ** (self.depth - 1), 2 * self.num_start_filters * 2 ** (self.depth - 1), kernel_size=1, padding=0),
            nn.ReLU()
        )
        self.logits = nn.Conv1d(self.num_start_filters, self.out_channels, 1, 1)


    def _create_encoder_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=5, padding=2),
            nn.ReLU()
        )

    def _create_decoder_block(self, in_channels, out_channels):
        return nn.ModuleList([nn.ConvTranspose1d(in_channels, in_channels//2, kernel_size=2, stride=2),
            nn.Conv1d(in_channels, out_channels, kernel_size=5, padding=2),
            nn.ReLU()])

    def _create_unet(self, in_channels, out_channels):
        for _ in range(self.depth):
            self.encoder.append(self._create_encoder_block(in_channels, out_channels))
            in_channels, out_channels = out_channels, out_channels*2

        out_channels = in_channels
        in_channels = in_channels * 2
        for _ in range(self.depth):
            self.decoder.append(self._create_decoder_block(in_channels, out_channels))
            in_channels, out_channels = out_channels, out_channels//2

    def forward(self, x):
        encoded = []
        for enc in self.encoder:
            x = enc(x)
            encoded.append(x)
            x = nn.MaxPool1d(kernel_size=2, stride=2)(x)

        x = self.bottleneck(x)  # Bottleneck layer

        for dec in self.decoder:
            ## Adding input with encoder concatenation
            enc_output = encoded.pop()
            x = dec[0](x)
            ## Pad the decoder output to match the encoder output
            diff = enc_output.shape[2] - x.shape[2]
            x = F.pad(x, (diff // 2, diff - diff // 2))
            x = torch.cat((enc_output, x), dim=1)
            x = dec[1](x)
            x = dec[2](x)
        ## Add softmax to logits
        # x = self.softmax(x)

        return self.logits(x)

input_channels = 6 
output_channels = 2
depth = 4
num_layers = 2

model = UNet1D(input_channels, output_channels, depth, num_layers)
print(model)

UNet1D(
  (encoder): ModuleList(
    (0): Sequential(
      (0): Conv1d(6, 32, kernel_size=(5,), stride=(1,), padding=(2,))
      (1): ReLU()
    )
    (1): Sequential(
      (0): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=(2,))
      (1): ReLU()
    )
    (2): Sequential(
      (0): Conv1d(64, 128, kernel_size=(5,), stride=(1,), padding=(2,))
      (1): ReLU()
    )
    (3): Sequential(
      (0): Conv1d(128, 256, kernel_size=(5,), stride=(1,), padding=(2,))
      (1): ReLU()
    )
  )
  (decoder): ModuleList(
    (0): ModuleList(
      (0): ConvTranspose1d(512, 256, kernel_size=(2,), stride=(2,))
      (1): Conv1d(512, 256, kernel_size=(5,), stride=(1,), padding=(2,))
      (2): ReLU()
    )
    (1): ModuleList(
      (0): ConvTranspose1d(256, 128, kernel_size=(2,), stride=(2,))
      (1): Conv1d(256, 128, kernel_size=(5,), stride=(1,), padding=(2,))
      (2): ReLU()
    )
    (2): ModuleList(
      (0): ConvTranspose1d(128, 64, kernel_size=(2,), stride=(2,))
      (1): C

In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

In [4]:
# Generate synthetic data
num_superpixels = 300
num_features = 6
synthetic_data = np.random.rand(num_superpixels, num_features)
synthetic_data = torch.tensor(synthetic_data, dtype=torch.float32)

#Reshape
synthetic_data = synthetic_data.unsqueeze(0).transpose(1, 2)

# Pass the synthetic data through the U-Net model
with torch.no_grad():
    output = model(synthetic_data)

print("Input shape:", synthetic_data.shape)
print("Output shape:", output.shape)

Input shape: torch.Size([1, 6, 300])
Output shape: torch.Size([1, 2, 300])


### Creating LightningModule 

In [5]:
class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        # BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = dice_loss
        
        return Dice_BCE

In [6]:
class CloudSegmentationModel(pl.LightningModule):
    def __init__(self, depth=3):
        super(CloudSegmentationModel, self).__init__()
        self.unet = UNet1D(in_channels=6, out_channels=1, depth=depth)
        
    def forward(self, x):
        return nn.Sigmoid()(self.unet(x))

    def accuracy_score(self, y_true, y_pred):
        y_true = y_true.cpu().detach().numpy()
        y_pred = y_pred.cpu().detach().numpy()
        y_pred = np.where(y_pred > 0.5, 1, 0)
        return accuracy_score(y_true, y_pred)

    def training_step(self, batch, batch_idx):
        superpixel, label = batch
        output = self(superpixel)
        loss = nn.BCELoss()(output, label)

        # Calculate accuracy
        predicted = torch.round(output)
        correct = (predicted == label).sum().item()
        total = label.size(0) * label.size(1) * label.size(2)
        accuracy = correct / total

        self.log('train_loss', loss, on_step=False, on_epoch=True)
        self.log('train_accuracy', accuracy, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        superpixel, label = batch
        output = self(superpixel)
        loss = nn.BCELoss()(output, label)

        # Calculate accuracy
        predicted = torch.round(output)
        correct = (predicted == label).sum().item()
        total = label.size(0) * label.size(1) * label.size(2)
        accuracy = correct / total

        self.log('val_loss', loss, on_step=False, on_epoch=True)
        self.log('val_accuracy', accuracy, on_step=False, on_epoch=True)


    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

In [7]:
model = CloudSegmentationModel()
print("Parameters: ",count_parameters(model))

with torch.no_grad():
    output = model(synthetic_data)

print("Input shape:", synthetic_data.shape)
print("Output shape:", output.shape)

Parameters:  386945
Input shape: torch.Size([1, 6, 300])
Output shape: torch.Size([1, 1, 300])


### Creating Dataset

In [28]:
patches,mask = data_utils.get_patch(path_to_folders_images = "../Dataset/Natural_False_Color/", path_to_folders_labels = "../Dataset/Entire_scene_gts/")

c:\Users\GP65\anaconda3\lib\site-packages\rasterio\__init__.py:333: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [29]:
X = []
y = []
for i,j in tqdm(list(zip(patches,mask))):
    try:
        output = data_utils.convert_image_array_to_slic_with_properties(i,j,n_segments=300) 

        ## Getting the X and y arrays
        X_array = np.array([list(list(i.values())[0]) + list(i.values())[1:] for i in output[1]])
        y_array = np.array(output[2])

        ## Normalizing the X_array columwise
        X_array[:,0] = X_array[:,0]/255
        X_array[:,1] = X_array[:,1]/255
        X_array[:,2] = X_array[:,2]/255
        X_array[:,3] = X_array[:,3]/512
        X_array[:,4] = X_array[:,4]/512
        X_array[:,5] = X_array[:,5]/1000



        ## Pad the X_array with -1 and y_array with 0 upto 300
        X_array = np.pad(X_array,((0,300-X_array.shape[0]),(0,0)),mode='constant',constant_values=-1)
        y_array = np.pad(y_array,(0,300-y_array.shape[0]),mode='constant',constant_values=0)

        ## Ordering
        ordering = mds.fit_transform(X_array[:,3:5]).reshape(-1)
        X_array = X_array[ordering.argsort()]
        y_array = y_array[ordering.argsort()].reshape(-1,1)

        ## Appending
        X.append(X_array)
        y.append(y_array)
        
    except KeyboardInterrupt:
        break
    except:
        pass

  0%|          | 0/1472 [00:00<?, ?it/s]

In [8]:
# np.save('../Dataset/X.npy', np.array(X))
# np.save('../Dataset/Y.npy', np.array(y))

X = np.load('../Dataset/X.npy')
y = np.load('../Dataset/Y.npy')

In [9]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x_item = torch.tensor(self.X[idx], dtype=torch.float32).T
        y_item = torch.tensor(self.y[idx], dtype=torch.float32).T
        return x_item, y_item

def create_dataloader(X, y, batch_size=32, shuffle=True):
    dataset = CustomDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

In [10]:
## Divide X and Y into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
train_loader = create_dataloader(X_train,y_train,batch_size=64,shuffle=True)
test_loader = create_dataloader(X_test,y_test,batch_size=64,shuffle=True)

## Training with PL

In [12]:
import matplotlib.pyplot as plt
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import TensorBoardLogger

In [24]:
# Initialize the model and trainer
segmentationModel = CloudSegmentationModel(depth=3)

In [25]:
logger = TensorBoardLogger('lightning_logs/', name='sgd_tagger')

In [26]:
checkpoint_callback = ModelCheckpoint(
    dirpath = './/model_checkpt/',
    filename = 'best-checkpoint',
    save_top_k=1,
    verbose = True,
    monitor='val_loss',
    mode='min'
)

In [27]:
trainer = Trainer(
    logger = logger,
    gpus=1 if torch.cuda.is_available() else None,
    max_epochs=100,
    callbacks=[EarlyStopping(monitor='val_loss', patience=20), checkpoint_callback]
)

# Train the model using the trainer
trainer.fit(segmentationModel, train_loader, test_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type   | Params
--------------------------------
0 | unet | UNet1D | 386 K 
--------------------------------
386 K     Trainable params
0         Non-trainable params
386 K     Total params
1.548     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Epoch 0: 100%|██████████| 34/34 [00:00<00:00, 34.01it/s, loss=0.618, v_num=4]

Epoch 0, global step 27: 'val_loss' reached 0.58780 (best 0.58780), saving model to 'D:\\Projects\\ComputerVision_CloudSegmentation\\Superpixel-UNET\\model_checkpt\\best-checkpoint-v5.ckpt' as top 1


Epoch 1: 100%|██████████| 34/34 [00:00<00:00, 46.01it/s, loss=0.514, v_num=4]

...

Epoch 66: 100%|██████████| 34/34 [00:00<00:00, 50.15it/s, loss=0.309, v_num=4]

Epoch 66, global step 1809: 'val_loss' was not in top 1


Epoch 67: 100%|██████████| 34/34 [00:00<00:00, 47.09it/s, loss=0.308, v_num=4]

Epoch 67, global step 1836: 'val_loss' was not in top 1


Epoch 67: 100%|██████████| 34/34 [00:00<00:00, 46.64it/s, loss=0.308, v_num=4]


## Training the module

In [103]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

segmentationModel = CloudSegmentationModel().to(device)

# train_loader = None # Train loader for our dataset
# test_loader = None # Test loader for our dataset

# loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(segmentationModel.parameters(), lr=0.001)

# Training loop
num_epochs = 100
segmentationModel.train() 

for epoch in tqdm(range(num_epochs)):
    running_loss = 0

    segmentationModel.train()
    for superpixel, label in train_loader:
        superpixel = superpixel.to(device)
        label = label.to(device)

        # Forward pass
        output = segmentationModel(superpixel)
        loss = criterion(output, label)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        ## Calculate accuracy
        predicted = torch.round(output)
        correct = (predicted == label).sum().item()
        total = label.size(0) * label.size(1) * label.size(2)
        accuracy = correct / total


    # Print epoch statistics
    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}')

    # Evaluation
    segmentationModel.eval()
    test_loss = 0

    with torch.no_grad():
        for superpixel, label in test_loader:
            superpixel = superpixel.to(device)
            label = label.to(device)

            output = segmentationModel(superpixel)
            test_loss += criterion(output, label).item()

            ## Calculate accuracy
            predicted = torch.round(output)
            correct = (predicted == label).sum().item()
            total = label.size(0) * label.size(1) * label.size(2)
            accuracy = correct / total


    test_loss /= len(test_loader)
    ## Print loss and accuracy
    print(f'Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}')


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/100], Loss: 0.6364, Accuracy: 0.6186
Test Loss: 0.5749, Accuracy: 0.6813
Epoch [2/100], Loss: 0.5424, Accuracy: 0.8075
Test Loss: 0.5319, Accuracy: 0.7332
Epoch [3/100], Loss: 0.5089, Accuracy: 0.7436
Test Loss: 0.5077, Accuracy: 0.7726
Epoch [4/100], Loss: 0.4980, Accuracy: 0.7191
Test Loss: 0.5190, Accuracy: 0.7041
Epoch [5/100], Loss: 0.4874, Accuracy: 0.7807
Test Loss: 0.5052, Accuracy: 0.7741
Epoch [6/100], Loss: 0.4865, Accuracy: 0.8059
Test Loss: 0.4932, Accuracy: 0.7659
Epoch [7/100], Loss: 0.4706, Accuracy: 0.7999
Test Loss: 0.4940, Accuracy: 0.6825
Epoch [8/100], Loss: 0.4623, Accuracy: 0.7862
Test Loss: 0.4570, Accuracy: 0.7324
Epoch [9/100], Loss: 0.4311, Accuracy: 0.7923
Test Loss: 0.4193, Accuracy: 0.8353
Epoch [10/100], Loss: 0.4135, Accuracy: 0.8057
Test Loss: 0.4009, Accuracy: 0.8276
Epoch [11/100], Loss: 0.3908, Accuracy: 0.8448
Test Loss: 0.3932, Accuracy: 0.8185
Epoch [12/100], Loss: 0.3822, Accuracy: 0.8141
Test Loss: 0.4353, Accuracy: 0.8000
Epoch [13/100